In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing, equ2gal
from qubic.lib.Qscene import QubicScene
from qubic.lib.MapMaking.FrequencyMapMaking.Qspectra_component import CMBModel

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

%matplotlib inline

In [ ]:
nside = 128
np.random.seed(6)

# Initialize QUBIC instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
# Build MPI communicator and qubic dict
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubicDict()
d.read_from_file(dictfilename)

center = equ2gal(d['RA_center'], d['DEC_center'])

d['nf_recon'] = 2
d['MultiBand'] = True
d['nside'] = nside

d['npointings'] = 10
d['synthbeam_kmax'] = 1
nf_sub = d['nf_sub']
d['synthbeam_fraction'] = 1

d['noiseless'] = True
d['photon_noise'] = False
d['use_synthbeam_fits_file'] = False

#d['nprocs_instrument'] = 1

In [ ]:
d

In [ ]:
# Build scanning strategy
sampling = get_pointing(d)

# Build scene
scene = QubicScene(d)

In [ ]:
# Number of sub-acquisitions
nsub_max = 14
nsub_list = np.arange(4, nsub_max + 1, 6)

In [ ]:
# Build CMB map
cl_cmb = CMBModel(None).give_cl_cmb(r=0, Alens=1)
cmb_map = hp.synfast(cl_cmb, nside, new=True, verbose=False).T

map_title = "I, Q, U = 1"

In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u

def parallactic_angle(ra, dec, time, location):
    """
    Compute the parallactic angle for a source at (ra, dec) given observation time and location.

    Parameters:
    -----------
    ra, dec : float
        Right ascension and declination of the source in degrees.
    time : str or astropy.time.Time
        Observation time in ISO format (or as an astropy Time object).
    location : astropy.coordinates.EarthLocation
        Observer's location.
    
    Returns:
    --------
    psi : float
        Parallactic angle in radians.
    """
    # Create a SkyCoord for the source in equatorial coordinates
    src = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs')
    
    # Define observation time as an astropy Time object
    obs_time = Time(time)
    
    # Define AltAz frame for the observer at given time and location
    altaz_frame = AltAz(obstime=obs_time, location=location)
    
    # Transform source to AltAz frame
    src_altaz = src.transform_to(altaz_frame)
    
    # Compute parallactic angle using Astropy's built-in function.
    # Astropy returns the parallactic angle in radians.
    psi = src_altaz.parallactic_angle.radian
    return psi

# Example usage:
# Observer's location (for example, Mauna Kea)
location = EarthLocation(lat=d["latitude"]*u.deg, lon=d["longitude"]*u.deg, height=500*u.m)

# Observation time
obs_time = d["time_obs"]  # ISO format

# Suppose you have a pixel with Galactic coordinates (l, b)
# First convert Galactic (l, b) to ICRS (RA, Dec)
gal_coord = SkyCoord(l=120*u.deg, b=45*u.deg, frame='galactic')
equ_coord = gal_coord.icrs
ra, dec = equ_coord.ra.deg, equ_coord.dec.deg

# Compute parallactic angle for that pixel
psi = parallactic_angle(ra, dec, obs_time, location)

def compute_galactic_stokes(Q0, U0, psi):
    """
    Compute the Galactic frame Stokes parameters (Q, U) that, after rotation by angle psi,
    yield uniform (Q0, U0) in the instrument frame.

    Parameters:
    Q0, U0 : float
        Desired uniform Stokes parameters in the instrument frame.
    psi : array-like
        Rotation angle (in radians) between the instrument's reference axis and the Galactic reference direction
        for each sky position.

    Returns:
    Q_gal, U_gal : array-like
        Stokes parameters in the Galactic frame for each sky position.
    """
    Q_gal = Q0 * np.cos(2 * psi) - U0 * np.sin(2 * psi)
    U_gal = Q0 * np.sin(2 * psi) + U0 * np.cos(2 * psi)
    return Q_gal, U_gal

# Example usage:

# Define the desired uniform Stokes parameters in the instrument frame
I0 = 1.0
Q0 = 0.1
U0 = 0.1

# Define the number of pixels in the HEALPix map (e.g., nside=64)
npix = hp.nside2npix(nside)

# Compute the Galactic frame Stokes parameters
Q_gal, U_gal = compute_galactic_stokes(Q0, U0, psi)

# Create HEALPix maps for Q and U in the Galactic frame
Q_map = np.zeros(npix)
U_map = np.zeros(npix)

Q_map[:] = Q_gal
U_map[:] = U_gal

# Save the maps to FITS files
hp.write_map('Q_galactic.fits', Q_map, overwrite=True)
hp.write_map('U_galactic.fits', U_map, overwrite=True)

print("Galactic frame Stokes parameter maps have been saved.")



In [ ]:
TOD_list = []
tod_list = []
freq_list = []

for insub in nsub_list:
    print(f"Nsub = {insub}")
    input_maps = np.ones((insub, hp.nside2npix(nside), 3)) * uniform_pol_map[None, ...]
    
    # Build acquisition operator
    Qacq = QubicMultiAcquisitions(d, insub, 2, sampling=sampling)
    
    # Build TOD
    tod = []
    _tod = []
    map_list = []
    for i in range(insub):
        P = Qacq.multiinstrument[i].get_projection_operator(sampling, scene)
        F = Qacq.multiinstrument[i].get_filter_operator()
        C = HealpixConvolutionGaussianOperator(fwhm=Qacq.allfwhm[i])
   
        tod.append(P(F(C(input_maps[i]))))
        _tod.append(P(C(input_maps[i])))
        
        del P, F, C

    TOD_list.append(np.sum(tod, axis=0))
    tod_list.append(_tod)
    freq_list.append(np.array(Qacq.allnus))
    
    del Qacq, tod, input_maps

In [ ]:
# # Define a list of colors to use (you can choose your own)
# colors = px.colors.qualitative.Plotly

# fig = make_subplots(rows=1, cols=3, subplot_titles=("I", "Q", "U"))

# for i in range(nsub_list.size):
#     # Use the same legend group and color for all three traces of the same i.
#     group_name = f"group{i}"
#     color = colors[i % len(colors)]
    
#     # First subplot: show legend
#     fig.add_trace(go.Scatter(
#         y=np.mean(TOD_list[i], axis=0)[:, 0],
#         name=f"Nsub = {nsub_list[i]}",
#         legendgroup=group_name,
#         line=dict(color=color, width=1),
#         showlegend=True
#     ), row=1, col=1)
    
#     # Second subplot: hide legend
#     fig.add_trace(go.Scatter(
#         y=np.mean(TOD_list[i], axis=0)[:, 1],
#         legendgroup=group_name,
#         line=dict(color=color, width=1),
#         showlegend=False
#     ), row=1, col=2)
    
#     # Third subplot: hide legend
#     fig.add_trace(go.Scatter(
#         y=np.mean(TOD_list[i], axis=0)[:, 2],
#         legendgroup=group_name,
#         line=dict(color=color, width=1),
#         showlegend=False
#     ), row=1, col=3)

# fig.update_layout(
#     title=f"TOD - Mean over detector - Nside = {nside}",
#     xaxis_title="Pointing",
#     yaxis_title="TOD",
#     width=2500,  # Width in pixels
#     height=600   # Height in pixels
# )
# fig.update_yaxes(tickformat=".1e")  # Example: show scientific notation with 2 decimals

# fig.show()

In [ ]:
# # Define a list of colors to use (you can choose your own)
# colors = px.colors.qualitative.Plotly

# fig = make_subplots(rows=1, cols=3, subplot_titles=("I", "Q", "U"))

# idet = 200

# for i in range(nsub_list.size):
#     # Use the same legend group and color for all three traces of the same i.
#     group_name = f"group{i}"
#     color = colors[i % len(colors)]
    
#     # First subplot: show legend
#     fig.add_trace(go.Scatter(
#         y=TOD_list[i][idet, :, 0],
#         name=f"Nsub = {nsub_list[i]}",
#         legendgroup=group_name,
#         line=dict(color=color, width=1),
#         showlegend=True
#     ), row=1, col=1)
    
#     # Second subplot: hide legend
#     fig.add_trace(go.Scatter(
#         y=TOD_list[i][idet, :, 1],
#         legendgroup=group_name,
#         line=dict(color=color, width=1),
#         showlegend=False
#     ), row=1, col=2)
    
#     # Third subplot: hide legend
#     fig.add_trace(go.Scatter(
#         y=TOD_list[i][idet, :, 2],
#         legendgroup=group_name,
#         line=dict(color=color, width=1),
#         showlegend=False
#     ), row=1, col=3)

# fig.update_layout(
#     title=f"TOD - Detector {idet}",
#     xaxis_title="Pointing",
#     yaxis_title="TOD",
# )
# fig.update_yaxes(tickformat=".1e")  # Example: show scientific notation with 2 decimals

# fig.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

isample = 0

for i in range(len(tod_list)):  
    color = plt.cm.jet(i/len(tod_list))
    ax1.plot(freq_list[i], np.mean(np.array(tod_list[i])[..., isample, 0], axis=1), '-x', label = f"Nsub = {nsub_list[i]}", color=color)
    ax2.plot(freq_list[i], np.mean(np.array(tod_list[i])[..., isample, 1], axis=1), '-x', color=color)
    ax3.plot(freq_list[i], np.mean(np.array(tod_list[i])[..., isample, 2], axis=1), '-x', color=color)
    
    ax1.hlines(np.mean(np.mean(np.array(tod_list[i])[..., isample, 0], axis=1)), 131.25, 245, color=color, linestyle='--', alpha=0.5)
    ax2.hlines(np.mean(np.mean(np.array(tod_list[i])[..., isample, 1], axis=1)), 131.25, 245, color=color, linestyle='--', alpha=0.5)
    ax3.hlines(np.mean(np.mean(np.array(tod_list[i])[..., isample, 2], axis=1)), 131.25, 245, color=color, linestyle='--', alpha=0.5)
ax1.set_xlabel('Frequency (GHz)')
ax1.set_ylabel("TOD")
ax1.set_title('I')
ax1.set_xlim(131.25, 245)

ax2.set_xlabel('Frequency (GHz)')
ax2.set_ylabel("TOD")
ax2.set_title('Q')
ax2.set_xlim(131.25, 245)

ax3.set_xlabel('Frequency (GHz)')
ax3.set_ylabel("TOD")
ax3.set_title('U')
ax3.set_xlim(131.25, 245)

fig.legend()
fig.suptitle(r'TOD - $H = P_{roj}C_{onv}$ - One Time Sample - Mean Over Detectors' + f' - Nside = {nside}')

In [ ]:
np.shape(tod_list[0][0])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

idet = 0

for i in range(len(tod_list)):  
    color = plt.cm.jet(i/len(tod_list))
    ax1.plot(freq_list[i], np.array(tod_list[i])[:, idet, isample, 0], '-x', label = f"Nsub = {nsub_list[i]}", color=color)
    ax2.plot(freq_list[i], np.array(tod_list[i])[:, idet, isample, 1], '-x', color=color)
    ax3.plot(freq_list[i], np.array(tod_list[i])[:, idet, isample, 2], '-x', color=color)
    
    ax1.hlines(np.mean(np.array(tod_list[i]), axis=0)[idet, isample, 0], 131.25, 245, color=color, linestyle='--', alpha=0.5)
    ax2.hlines(np.mean(np.array(tod_list[i]), axis=0)[idet, isample, 1], 131.25, 245, color=color, linestyle='--', alpha=0.5)
    ax3.hlines(np.mean(np.array(tod_list[i]), axis=0)[idet, isample, 2], 131.25, 245, color=color, linestyle='--', alpha=0.5)
ax1.set_xlabel('Frequency (GHz)')
ax1.set_ylabel("TOD")
ax1.set_title('I')
ax1.set_xlim(131.25, 245)

ax2.set_xlabel('Frequency (GHz)')
ax2.set_ylabel("TOD")
ax2.set_title('Q')
ax2.set_xlim(131.25, 245)

ax3.set_xlabel('Frequency (GHz)')
ax3.set_ylabel("TOD")
ax3.set_title('U')
ax3.set_xlim(131.25, 245)

fig.legend()
fig.suptitle(r'TOD - $H = P_{roj}C_{onv}$ - One Time Sample' + f' - Detector {idet} - Nside = {nside} - Map : {map_title}')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

for i in range(len(TOD_list)):    
    ax1.plot(np.mean(TOD_list[i][..., 0], axis=0), '-x', label = f"Nsub = {nsub_list[i]}")
    ax2.plot(np.mean(TOD_list[i][..., 1], axis=0), '-x')
    ax3.plot(np.mean(TOD_list[i][..., 2], axis=0), '-x')

ax1.set_xlabel('Pointing')
ax1.set_ylabel("TOD")
ax1.set_title('I')

ax2.set_xlabel('Pointing')
ax2.set_ylabel("TOD")
ax2.set_title('Q')

ax3.set_xlabel('Pointing')
ax3.set_ylabel("TOD")
ax3.set_title('U')

fig.legend()
fig.suptitle(r'TOD - $H = P_{roj}F_{ilter}C_{onv}$ - Mean Over Detectors - ' + f' - Nside = {nside} - Map : {map_title}')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

for i in range(len(TOD_list)):    
    ax1.plot(TOD_list[i][idet, :, 0], '-x', label = f"Nsub = {nsub_list[i]}")
    ax2.plot(TOD_list[i][idet, :, 1], '-x')
    ax3.plot(TOD_list[i][idet, :, 2], '-x')

ax1.set_xlabel('Pointing')
ax1.set_ylabel("TOD")
ax1.set_title('I')

ax2.set_xlabel('Pointing')
ax2.set_ylabel("TOD")
ax2.set_title('Q')

ax3.set_xlabel('Pointing')
ax3.set_ylabel("TOD")
ax3.set_title('U')

fig.legend()
fig.suptitle(r'TOD - $H = P_{roj}F_{ilter}C_{onv}$ - Detector' + f' {idet} - Nside = {nside} - Map : {map_title}')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

for i in range(len(TOD_list)):
    rel_diff = (TOD_list[i] - TOD_list[-1]) / TOD_list[-1]
    
    ax1.plot(nsub_list[i], np.mean(np.abs(rel_diff)[:, isample, 0]), '.')
    ax2.plot(nsub_list[i], np.mean(np.abs(rel_diff)[:, isample, 1]), '.')
    ax3.plot(nsub_list[i], np.mean(np.abs(rel_diff)[:, isample, 2]), '.')

ax1.set_xlabel('Nsub')
ax1.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax1.set_title('I')

ax2.set_xlabel('Nsub')
ax2.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax2.set_title('Q')

ax3.set_xlabel('Nsub')
ax3.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax3.set_title('U')

fig.suptitle(r'Convergence of the TOD sub-operators - $H = P_{roj}F_{ilter}C_{onv}$ - Mean Over Detectors' + f' - Nside = {nside} - Map : {map_title}')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

for i in range(len(TOD_list)):
    rel_diff = (TOD_list[i] - TOD_list[-1]) / TOD_list[-1]
    
    ax1.plot(nsub_list[i], np.mean(np.abs(rel_diff)[idet, isample, 0]), '.')
    ax2.plot(nsub_list[i], np.mean(np.abs(rel_diff)[idet, isample, 1]), '.')
    ax3.plot(nsub_list[i], np.mean(np.abs(rel_diff)[idet, isample, 2]), '.')

ax1.set_xlabel('Nsub')
ax1.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax1.set_title('I')

ax2.set_xlabel('Nsub')
ax2.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax2.set_title('Q')

ax3.set_xlabel('Nsub')
ax3.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
ax3.set_title('U')

fig.suptitle(r'Convergence of the TOD sub-operators - $H = P_{roj}F_{ilter}C_{onv}$ - Detector' + f' {idet} - Nside = {nside} - Map : {map_title}')